# Improved Asymptotic Iteration Method (IAIM)

Main idea, expand by Taylor series the coefficients $\lambda_n$ and $s_n$, evaluated at point $y = y_0$, as: \
\
$ \lambda_n = \sum_{k}  c_n^k (y-y_0)^k $ \
$ s_n = \sum_{k} d_n^k (y-y_0)^k  $

There exists a recursion relation for the coefficients: \
\
$ c_n^i = (i+1)c_{n-1}^{i+1} + d_{n-1}^i + \sum_k c_0^kc_{n-1}^{i-k} $ \
$ d_n^i = (i+1)d_{n-1}^{i+1} \sum_k d_0^k c_{n-1}^{i-k} $

And the quantization condition becomes: $\qquad \delta_n^0= c_{n-1}^0 - d_{n-1}^0c_n^0 = 0 $

Parameters: $\qquad \lambda_0 = \dfrac{2y(1-iw)}{1-y^2} \qquad s_0 = \dfrac{1-2iw-2w^2}{2(1-y^2)} $

Begin by importing necessary libraries

In [132]:
#USEFUL LIBRATIES
import numpy as np
#import matplotlib.pyplot as plt
from sympy import *

#NUMBER OF ITERATIONS TO PERFORM
N = 4

#SYMBOLIC VARIABLES DEFINITION
y = symbols("y", real=True)
w = symbols("\omega")

#INITIAL PARAMETERS
l0 = (2*y*(1-I*w))/(1-y**2)
s0 = (1-2*I*w-2*w**2)/(2*(1-y**2))

We must compute the series expansion of $\lambda_0$ and $s_0$ up until order $i=n$

In [133]:
#FUNCTION THAT GETS COEFFICIENTS AS LIST FROM SERIES EXPANSION
def get_coeff(a):
    
    coeff = []
    coeff = np.append(coeff, a.subs(y,0))
    for i in range(1,N):
        coeff = np.append(coeff, a.coeff(y**i))

    return coeff

In [134]:
l0_series = series(l0,y,0,N).removeO()
#l0_series

In [135]:
s0_series = series(s0,y,0,N).removeO()
#s0_series

In [136]:
l0_coeff = get_coeff(l0_series)
s0_coeff = get_coeff(s0_series)

The coefficients of these series expansions are columns of the matrices $C$ and $D$. Once we have the first column (obtained from $\lambda_0$ and $s_0$, we move on to the next)

In [137]:
C = np.zeros((N,N),dtype=object)
D = np.zeros((N,N),dtype=object)

In [138]:
for i in range(N):
    C[i,0] = l0_coeff[i]
for i in range(N):
    D[i,0] = s0_coeff[i]

pprint(Matrix(C))

⎡       0         0  0  0⎤
⎢                        ⎥
⎢-2⋅ⅈ⋅\omega + 2  0  0  0⎥
⎢                        ⎥
⎢       0         0  0  0⎥
⎢                        ⎥
⎣-2⋅ⅈ⋅\omega + 2  0  0  0⎦


In [120]:
#pprint(Matrix(D))
D[0,0]

In [128]:
for n in range(0,N-1): #COLUMNS
    for i in range(0,N-1): #ROWS
        D[i,n+1] = (i+1)*D[i+1,n]
        C[i,n+1] = (i+1)*C[i+1,n]+D[i,n]
        for k in range(0,i): #SUM INTERIOR
            C[i,n+1] = C[i,n] + C[k,0]*C[i-k,n]
            D[i,n+1] = D[i,n] + D[k,0]*C[i-k,n]
    
#pprint(Matrix(C))
C[0,1]

In [126]:
d = D[0,N-1]*C[0,N-2] - D[0,N-2]*C[0,N-1]
d

In [127]:
solve(d,w)